In [1]:
import math
import random

In [3]:
class Value:
    
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data**(other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + -other
    
    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1) 
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()    

In [4]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        
    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [5]:
n = MLP(3, [4,4,1])

xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]

In [6]:
for epochs in range(1000):
    
    ypred = [n(x) for x in xs]
    loss = sum((y_hat - ygt)**2 for ygt, y_hat in zip(ys, ypred))
    
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    for p in n.parameters():
        p.data += -0.01 * p.grad
    
    print(epochs, ":", loss.data)

0 : 8.118371436438542
1 : 7.5255890997399
2 : 6.997876738178947
3 : 6.512499947790724
4 : 6.046639274773419
5 : 5.5742936341263025
6 : 5.0838002756833784
7 : 4.61315840284715
8 : 4.228290235270426
9 : 3.9360744784940174
10 : 3.702256340090939
11 : 3.5002865368245013
12 : 3.3151346607310668
13 : 3.1378698099326656
14 : 2.9629027387855134
15 : 2.786983346229266
16 : 2.6088904306975347
17 : 2.429291662757675
18 : 2.2504571092815815
19 : 2.075658255945302
20 : 1.9083188977720444
21 : 1.7512138595266047
22 : 1.6060280002825196
23 : 1.4733578783205545
24 : 1.352991468525994
25 : 1.2442415346261555
26 : 1.1462019116182958
27 : 1.057902634225207
28 : 0.9783901948200103
29 : 0.9067645978720528
30 : 0.842194587973613
31 : 0.7839224222879766
32 : 0.7312635554769098
33 : 0.6836036977495811
34 : 0.6403944494224758
35 : 0.6011481865976407
36 : 0.5654326295574461
37 : 0.5328653887497596
38 : 0.5031086892484448
39 : 0.4758644033673537
40 : 0.4508694666051369
41 : 0.4278917116394044
42 : 0.406726126440

In [7]:
ypred

[Value(data=0.9650828646001821),
 Value(data=-0.9871167491485532),
 Value(data=-0.9593646414047038),
 Value(data=0.9696039611727914)]